In [1]:
from magpie import grasp as gt
import rtde_control
import rtde_receive
from magpie.motor_code import Motors
from magpie import ur5 as ur5
import time
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import importlib
importlib.reload(gt)
importlib.reload(ur5)

<module 'magpie.ur5' from '/home/will/MAGPIE/magpie/ur5.py'>

In [3]:
# tmat = np.array([[ 0.91574736, -0.35602909,  0.18614527, -0.02848649],
#        [-0.40073104, -0.77640133,  0.48643151,  0.14332736],
#        [ 0.02866033,  0.52004256,  0.85365937,  0.33750796],
#        [ 0.        ,  0.        ,  0.        ,  1.        ]])

tmat = np.array([[ 0.99398525,  0.10940962,  0.00478187, -0.02280831],
        [-0.10899759,  0.99258961, -0.0537159 , -0.03559815],
        [-0.01062347,  0.0528716 ,  0.99854481,  0.23497098],
        [ 0.        ,  0.        ,  0.        ,  1.        ]])

mmc = copy.deepcopy(tmat[:3, 3])
# grasp_pose = [mmc[1], -mmc[0], mmc[2]]
grasp_pose = mmc
tmat[:3, 3] = [0, 0, 0]
homePose = None

In [4]:
# orient gripper in place OR move gripper to cartesian pos
sleepRate = 1.5
ur = ur5.UR5_Interface()
try:
    ur.start()
    homePose = ur.get_tcp_pose()
    currentPose = ur.getPose()
    desiredPose = np.matmul(np.array(currentPose), tmat)
    print("Desired Pose: ", desiredPose)
    # in-place (or best attempt) re-orientation
    ur.moveL(desiredPose)
    ## move to cartesian pos
    # gt.move_to(grasp_pose, ur)    
    print("Done moving to block")
    time.sleep(sleepRate)
    time.sleep(sleepRate)
    ur.stop()
except Exception as e:
    ur.stop()
    raise(e)

Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
Desired Pose:  [[-0.986  0.157 -0.049 -0.027]
 [ 0.163  0.97  -0.181 -0.582]
 [ 0.019 -0.187 -0.982  0.229]
 [ 0.     0.     0.     1.   ]]
Done moving to block


In [7]:
# move home
ur = ur5.UR5_Interface()
try:
    ur.start()
    time.sleep(sleepRate)
    ur.moveL(homePose)
    time.sleep(sleepRate)
    ur.stop()
except Exception as e:
    ur.stop()

Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 


In [6]:
# orient and move gripper to cartesian pos
sleepRate = 1.5
ur = ur5.UR5_Interface()
try:
    servoPort = "/dev/ttyACM0"
    ur.start()
    time.sleep(sleepRate)
    homePose = ur.get_tcp_pose()
    dX,dY,dZ = gt.get_world_frame(grasp_pose, ur)
    

    '''
    stopping point 2/15:
    this works so far
    take home pos
    add translation on the gripper frame to the goal x, y
    apply tmat to the goal
    move to goal (translates on x, y and orients at the same time)
    ## does this address the fact that orientation is not achievable in standstill?

    then: take orig Z (from original gripper frame): [0, 0, dZ]
    transform by tmat (the current orientation of the gripper)
    add transformed dZ to the goal position
    move
    works!
    '''
    goal1 = copy.deepcopy(homePose)
    goal1 = np.array(goal1)
    goal1[:3, 3] += [dX, dY, 0]
    goal1 = np.array(goal1) @ tmat
    ur.moveL(goal1)
    time.sleep(sleepRate)

    # todo: take currentPose and validate orientation?
    goal2 = goal1
    z_off = -0.00
    posd = [0, 0, dZ - z_off] @ tmat[:3, :3]
    # posd = [0, 0, dZ - z_off]
    goal2 = np.array(goal2)
    goal2[:3, 3] += posd
    ur.moveL(goal2)
    time.sleep(sleepRate)
    '''
    end of working section
    '''
    ur.stop()
except Exception as e:
    ur.stop()
    raise(e)

Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
[[-0.964  0.266 -0.03  -0.027]
 [ 0.268  0.955 -0.132 -0.582]
 [-0.006 -0.135 -0.991  0.229]
 [ 0.     0.     0.     1.   ]]
P_goal:
[ 0.008 -0.06  -0.149]


In [5]:
# orient and move gripper to cartesian pos
sleepRate = 1.5
ur = ur5.UR5_Interface()
try:
    servoPort = "/dev/ttyACM0"
    gripperController = Motors(servoPort)
    gripperController.torquelimit(600) # used to be 600
    gripperController.speedlimit(100)
    ur.start()
    ur.gripper = gripperController
    time.sleep(sleepRate)

    homePose = ur.get_tcp_pose()
    dX,dY,dZ = gt.get_world_frame(grasp_pose, ur)
    '''
    stopping point 2/15:
    this works so far
    take home pos
    add translation on the gripper frame to the goal x, y
    apply tmat to the goal
    move to goal (translates on x, y and orients at the same time)
    ## does this address the fact that orientation is not achievable in standstill?

    then: take orig Z (from original gripper frame): [0, 0, dZ]
    transform by tmat (the current orientation of the gripper)
    add transformed dZ to the goal position
    move
    works!
    '''
    goal1 = copy.deepcopy(homePose)
    goal1 = np.array(goal1)
    goal1[:3, 3] += [dX, dY, 0]
    goal1 = np.array(goal1) @ tmat
    ur.moveL(goal1)
    time.sleep(sleepRate)

    # todo: take currentPose and validate orientation?
    goal2 = goal1
    z_off = -0.00
    posd = [0, 0, dZ - z_off] @ tmat[:3, :3]
    goal2 = np.array(goal2)
    goal2[:3, 3] += posd
    ur.moveL(goal2)
    time.sleep(sleepRate)
    '''
    end of working section
    '''

    print("Done moving to block")
    # ur.closeGripper(9) 
    time.sleep(sleepRate)
    # gt.moveBackFromObj(homePose, ur)
    # gt.grabStrat1(grasp_pose)
    ur.moveL(homePose)
    time.sleep(sleepRate)
    # ur.openGripper()
    ur.stop()
except Exception as e:
    gripperController.openGripper()
    gripperController.disconnect()
    ur.stop()
    raise(e)

UR5 Interface Established


AttributeError: 'NoneType' object has no attribute 'moveL'